# FIFA World Cup 2022 Tweets - Limpieza y Preprocesamiento con NLTK

In [1]:
#%pip install nltk
#%pip install numpy
%pip install emoji
#%pip install pandas
%pip install regex


   ---------------------------------------- 0.0/590.6 kB ? eta -:--:--
   ---------------------------------------- 590.6/590.6 kB 3.0 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.



In [1]:
import pandas as pd # Importar pandas y asignarle el alias pd
import regex as re
import emoji
import sys          # Importar sys
import os
import nltk

#  Descargar recursos de NLTK

In [6]:
# --- Descargar recursos de NLTK (solo necesitas hacer esto una vez) ---
# Estos son necesarios para la tokenización y remoción de stopwords
print("Verificando/Descargando recursos de NLTK ('punkt', 'stopwords')...")
try:
    nltk.data.find('tokenizers/punkt')
except (nltk.downloader.DownloadError, LookupError):
    print("Recurso 'punkt' de NLTK no encontrado. Intentando descargar...")
    nltk.download('punkt')

try:
    nltk.data.find('corpora/stopwords')
except (nltk.downloader.DownloadError, LookupError):
    print("Recurso 'stopwords' de NLTK no encontrado. Intentando descargar...")
    nltk.download('stopwords')



Verificando/Descargando recursos de NLTK ('punkt', 'stopwords')...


# Cargar las stopwords una vez


In [11]:
stop_words = set(nltk.corpus.stopwords.words('english'))
print("Recursos de NLTK 'punkt' y 'stopwords' verificados/descargados.")


Recursos de NLTK 'punkt' y 'stopwords' verificados/descargados.


#  Configuración

In [12]:
# --- Configuración ---
ruta_base = os.path.dirname(os.getcwd())
input_folder = 'data'
input_filename = 'fifa_world_cup_2022_tweets.csv'
input_csv_file = os.path.join(ruta_base,input_folder, input_filename)


output_folder = os.path.join(ruta_base,'data_processed') 
output_filename = 'fifa_tweets_clean.csv'
output_csv_file = os.path.join(output_folder, output_filename)

original_columns_expected = [
    'Date Created',
    'Number of Likes',
    'Source of Tweet',
    'Tweet',
    'Sentiment'
]

sentiment_mapping = {
    'negative': 0,
    'neutral': 1,
    'positive': 2
}

#  Funciones de procesamiento

In [19]:
def extract_hashtags(tweet):
    """Extrae hashtags de un tweet."""
    if pd.isna(tweet):
        return []
    tweet_str = str(tweet)
    hashtags = re.findall(r'#(\w+)', tweet_str)
    return hashtags

def get_emoji_descriptions(tweet):
    """Convierte emojis en texto descriptivo (ej: ❤️ -> red heart)."""
    if pd.isna(tweet):
        return []
    tweet_str = str(tweet)
    emoji_list_found = emoji.emoji_list(tweet_str)
    descriptions = []
    for emo in emoji_list_found:
        description = emoji.demojize(emo['emoji'], delimiters=("", "")).lower()
        descriptions.append(description)
    return descriptions

def extract_emoji_chars(tweet):
    """Extrae solo los caracteres de los emojis de un tweet."""
    if pd.isna(tweet):
        return []
    tweet_str = str(tweet)
    emoji_list_found = emoji.emoji_list(tweet_str)
    chars = [emo['emoji'] for emo in emoji_list_found]
    return chars


def clean_tweet(tweet):
    """
    Limpia un tweet: elimina URLs, convierte menciones, emojis y hashtags a tokens especiales,
    y elimina caracteres generales permitiendo letras de cualquier idioma, números, espacios y guiones bajos.
    """
    if pd.isna(tweet):
        return ""

    text = str(tweet)

    text = re.sub(r'http[s]?://\S+', '', text)
    text = re.sub(r'pic.twitter.com/\S+', '', text)
    text = re.sub(r't.co/\S+', '', text)

    text = re.sub(r'\B@(\w+)', r' _MENTION_\1_ ', text)


    emoji_list = emoji.emoji_list(text)
    for emo in reversed(emoji_list):
        start, end = emo['match_start'], emo['match_end']
        emo_char = emo['emoji']
        # Obtener descripción demojizada, reemplazar espacios por guiones bajos
        # Lowercase la descripción
        desc = emoji.demojize(emo_char, delimiters=("", "")).replace(" ", "_").lower()
        # Crear token especial con espacios alrededor para ayudar a la tokenización
        special_token = f" _EMOJI_{desc}_ "
        text = text[:start] + special_token + text[end:]


    text = re.sub(r'\B#(\w+)', lambda m: f" _HASHTAG_{m.group(1).lower()}_ ", text)


    text = re.sub(r'[^\p{L}0-9\s_]+', '', text, flags=re.UNICODE)

    text = re.sub(r'\s+', ' ', text).strip()

    text = text.lower()

    return text

import pandas as pd
import nltk
from nltk.corpus import stopwords


try:
    standard_stopwords = set(stopwords.words('english'))
except LookupError:
    print("Error: NLTK stopwords no encontradas. Descargando...")
    nltk.download('stopwords')
    standard_stopwords = set(stopwords.words('english'))



negation_words_lower = {
    "no", "not", "never", "none", "nobody", "nothing", "nowhere",
    "hardly", "scarcely", "barely", "seldom", "rarely",
    "don't", "doesn't", "didn't", "isn't", "aren't", "wasn't", "weren't",
    "haven't", "hasn't", "hadn't", "won't", "wouldn't", "can't", "cannot",
    "couldn't", "shouldn't", "mightn't", "mustn't",
    "n't"
}


standard_stopwords_lower = set(word.lower() for word in standard_stopwords)

custom_stopwords_lower = standard_stopwords_lower - negation_words_lower

print("DEBUG: custom_stopwords_lower creado. Contiene 'not'?", 'not' in custom_stopwords_lower)
print("DEBUG: custom_stopwords_lower creado. Contiene 'n\'t'?", 'n\'t' in custom_stopwords_lower)
print("DEBUG: custom_stopwords_lower creado. Contiene 'no'?", 'no' in custom_stopwords_lower)


# --- Definir la función de procesamiento de texto con DEBUGGING ---
def process_text_for_ml(text):
    """
    Procesa texto limpio: aplica tokenización, remoción de stopwords (excluyendo negaciones),
    y filtra palabras cortas, conservando tokens especiales de mencion, emoji y hashtag.

    Argumentos:
        text (str): La cadena de texto limpia (proveniente de 'test_clean').

    Retorna:
        list: Una lista de tokens procesados. Retorna lista vacía para inputs no válidos.
    """


    if pd.isna(text) or not isinstance(text, str) or text.strip() == "":
        # print("DEBUG: Input text is invalid, returning [].")
        return []


    tokens = nltk.tokenize.word_tokenize(text)

    mention_prefix = "_mention_"
    emoji_prefix = "_emoji_"
    hashtag_prefix = "_hashtag_"

    processed_tokens = []
    for word in tokens:
        word_lower = word.lower() # Convertir a minúsculas para comparación



        # Condición 1: Verificar si es stopword (excluyendo negaciones)
        is_stop = word_lower in custom_stopwords_lower



        # Condición 2 & 3: Verificar longitud o si es token especial
        is_special_token = word.startswith(mention_prefix) or word.startswith(emoji_prefix) or word.startswith(hashtag_prefix)
        passes_length_check = len(word) > 1


        # Lógica de filtrado combinada
        if not is_stop and (passes_length_check or is_special_token):
            processed_tokens.append(word)

    return processed_tokens

DEBUG: custom_stopwords_lower creado. Contiene 'not'? False
DEBUG: custom_stopwords_lower creado. Contiene 'n't'? False
DEBUG: custom_stopwords_lower creado. Contiene 'no'? False


#  Procesamiento principal: Carga y Preprocesamiento


In [20]:
print(f"Cargando dataset desde '{input_csv_file}'...")

try:
    df = pd.read_csv(
        input_csv_file,
        encoding='utf-8',
        encoding_errors='replace',
        engine='python',
        on_bad_lines='skip'
    )

    print(f"Dataset cargado exitosamente. Número de filas: {len(df)}")

except FileNotFoundError:
    print(f"Error: El archivo '{input_csv_file}' no fue encontrado.")
    print(f"Verifica la ruta: '{input_csv_file}'. Asegúrate de que el archivo CSV esté en la carpeta '{input_folder}' dentro del directorio donde ejecutas el script, o ajusta la ruta.")
    sys.exit(1)
except Exception as e:
    print(f"Error inesperado al cargar el archivo CSV: {e}")
    sys.exit(1)

if not all(col in df.columns for col in original_columns_expected):
    missing = [col for col in original_columns_expected if col not in df.columns]
    print(f"Error: Faltan columnas requeridas en el dataset de entrada: {missing}")
    print(f"Columnas disponibles: {df.columns.tolist()}")
    sys.exit(1)

df['Tweet'] = df['Tweet'].fillna('').astype(str) # Asegurarse de que la columna Tweet es string
df['Sentiment'] = df['Sentiment'].fillna('unknown').astype(str) # Asegurarse de que la columna Sentiment es string

print("Iniciando procesamiento mejorado de tweets...")

# --- Aplicar las funciones de preprocesamiento ---

df['test_clean'] = df['Tweet'].apply(clean_tweet)
print("Columna 'test_clean' (texto limpio con tokens especiales) creada.")

df['processed_tokens'] = df['test_clean'].apply(process_text_for_ml)
print("Columna 'processed_tokens' (lista de tokens procesados) creada.")

df['sentiment_label'] = df['Sentiment'].map(sentiment_mapping)

df['sentiment_label'] = df['sentiment_label'].fillna(-1).astype(int)
print("Columna 'sentiment_label' (numérica) creada.")

print("Procesamiento mejorado de tweets completado.")

print(f"\n--- Guardando dataset procesado para ML ---")

Cargando dataset desde 'c:\Users\oscar\Documents\Semestre6\discretas\ti2-2025-1-lora_team\data\fifa_world_cup_2022_tweets.csv'...
Dataset cargado exitosamente. Número de filas: 22524
Iniciando procesamiento mejorado de tweets...


Columna 'test_clean' (texto limpio con tokens especiales) creada.
Columna 'processed_tokens' (lista de tokens procesados) creada.
Columna 'sentiment_label' (numérica) creada.
Procesamiento mejorado de tweets completado.

--- Guardando dataset procesado para ML ---


#  Seleccionar y guardar las columnas procesadas ---


In [21]:
print(f"\n--- Guardando dataset procesado para ML ---")

final_output_columns_order = [
    'Tweet',
    'Sentiment',
    'test_clean',
    'processed_tokens', 
    'sentiment_label'    
]


final_output_columns_existing = [col for col in final_output_columns_order if col in df.columns]

if len(final_output_columns_existing) < len(final_output_columns_order):
    missing = [col for col in final_output_columns_order if col not in df.columns]
    print(f"Advertencia: Las siguientes columnas esperadas para guardar NO se encontraron en el dataset: {missing}")
    print("Guardando solo las columnas disponibles en la lista.")
   

try:
   
    os.makedirs(output_folder, exist_ok=True)
    print(f"Carpeta de salida '{output_folder}' asegurada.")
except Exception as e:
    print(f"Error al crear la carpeta de salida '{output_folder}': {e}")
    sys.exit(1) 

try:
    df[final_output_columns_existing].to_csv(output_csv_file, index=False, encoding='utf-8')
    print(f"Dataset guardado exitosamente en '{output_csv_file}'.")
except Exception as e:
    print(f"Error al guardar el archivo de salida '{output_csv_file}': {e}")
    print("Asegúrate de tener permisos de escritura en la carpeta de salida.")
    sys.exit(1) 

print("\nScript de preprocesamiento para ML completado.")
print(f"Columnas en el archivo de salida: {final_output_columns_existing}")

print(df[['Tweet', 'test_clean', 'processed_tokens', 'Sentiment', 'sentiment_label']].head())


--- Guardando dataset procesado para ML ---
Carpeta de salida 'c:\Users\oscar\Documents\Semestre6\discretas\ti2-2025-1-lora_team\data_processed' asegurada.
Dataset guardado exitosamente en 'c:\Users\oscar\Documents\Semestre6\discretas\ti2-2025-1-lora_team\data_processed\fifa_tweets_clean.csv'.

Script de preprocesamiento para ML completado.
Columnas en el archivo de salida: ['Tweet', 'Sentiment', 'test_clean', 'processed_tokens', 'sentiment_label']
                                               Tweet  \
0  What are we drinking today @TucanTribe \n@MadB...   
1  Amazing @CanadaSoccerEN  #WorldCup2022 launch ...   
2  Worth reading while watching #WorldCup2022 htt...   
3  Golden Maknae shinning bright\n\nhttps://t.co/...   
4  If the BBC cares so much about human rights, h...   

                                          test_clean  \
0  what are we drinking today _mention_tucantribe...   
1  amazing _mention_canadasocceren_ _hashtag_worl...   
2  worth reading while watching _hashtag_

# Guardando dataset procesado EDA



In [26]:
print(f"\n--- Guardando dataset procesado para ML y EDA ---")


output_folder = os.path.join(ruta_base,'data_processed') 
output_filename_eda = 'fifa_tweets_clean_for_EDA.csv'
output_csv_file_eda = os.path.join(output_folder, output_filename_eda)

final_output_columns_order_eda = [
    'Number of Likes',
    'Tweet',
    'Sentiment', 
    'test_clean', 
    'processed_tokens', 
    'hastag',
    'emoji_chars', 
    'sentiment_label'   
]


if 'hastag' not in df.columns:
     df['hastag'] = df['Tweet'].astype(str).apply(extract_hashtags) 
     print("Columna 'hastag' generada para el output CSV.")
if 'emoji_chars' not in df.columns:
     df['emoji_chars'] = df['Tweet'].astype(str).apply(extract_emoji_chars) 
     print("Columna 'emoji_chars' generada para el output CSV.")

final_output_columns_existing = [col for col in final_output_columns_order_eda if col in df.columns]

if len(final_output_columns_existing) < len(final_output_columns_order_eda):
    missing = [col for col in final_output_columns_order_eda if col not in df.columns]
    print(f"Advertencia: Las siguientes columnas esperadas para guardar NO se encontraron en el dataset: {missing}")
    print("Guardando solo las columnas disponibles en la lista.")

try:
    
    os.makedirs(output_folder, exist_ok=True)
    print(f"Carpeta de salida '{output_folder}' asegurada.")
except Exception as e:
    print(f"Error al crear la carpeta de salida '{output_folder}': {e}")
    sys.exit(1) 

try:
    df[final_output_columns_existing].to_csv(output_csv_file_eda, index=False, encoding='utf-8')
    print(f"Dataset guardado exitosamente en '{output_csv_file_eda}'.")
except Exception as e:
    print(f"Error al guardar el archivo de salida '{output_csv_file_eda}': {e}")
    print("Asegúrate de tener permisos de escritura en la carpeta de salida.")
    sys.exit(1) 

print("\nScript de preprocesamiento mejorado completado. Archivo listo para EDA y modelado.")
print(f"Columnas en el archivo de salida: {final_output_columns_existing}")


print("\nEjemplo de filas procesadas (incluyendo hastag y emoji_chars):")
print(df[['Tweet', 'test_clean', 'processed_tokens', 'hastag', 'emoji_chars', 'Sentiment', 'sentiment_label']].head())


--- Guardando dataset procesado para ML y EDA ---
Carpeta de salida 'c:\Users\oscar\Documents\Semestre6\discretas\ti2-2025-1-lora_team\data_processed' asegurada.
Dataset guardado exitosamente en 'c:\Users\oscar\Documents\Semestre6\discretas\ti2-2025-1-lora_team\data_processed\fifa_tweets_clean_for_EDA.csv'.

Script de preprocesamiento mejorado completado. Archivo listo para EDA y modelado.
Columnas en el archivo de salida: ['Number of Likes', 'Tweet', 'Sentiment', 'test_clean', 'processed_tokens', 'hastag', 'emoji_chars', 'sentiment_label']

Ejemplo de filas procesadas (incluyendo hastag y emoji_chars):
                                               Tweet  \
0  What are we drinking today @TucanTribe \n@MadB...   
1  Amazing @CanadaSoccerEN  #WorldCup2022 launch ...   
2  Worth reading while watching #WorldCup2022 htt...   
3  Golden Maknae shinning bright\n\nhttps://t.co/...   
4  If the BBC cares so much about human rights, h...   

                                          test_clea